In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
file_name = 'songs.txt'
with open(file_name, encoding="utf8") as f:
    text = f.read()

In [3]:
print(text[:100])

Doesn't take much to make me happy
And make me smile with glee 
Never never will I feel discouraged


In [4]:
# a tuple containg each unique character
chars = tuple(set(text))
# dictionary having integers as key - chars as values
int2char = dict(enumerate(chars))
# dictionary having chars as key - integers as values
char2int = dict((v,k) for k,v in int2char.items()) # https://stackoverflow.com/questions/1031851/how-do-i-exchange-keys-with-values-in-a-dictionary
# convert the whole text into an array of numbers with the corresponding integer for each character
encoded = np.array([char2int[ch] for ch in text])
print(encoded[:100])

[67  6  4 41 87 46 24 19 24  7 52  4 19 25 64 51 26 19 24  6 19 25  7 52
  4 19 25  4 19 26  7 77 77 72 65 88 87 97 19 25  7 52  4 19 25  4 19 41
 25 60 14  4 19 75 60 24 26 19 61 14  4  4 19 65 98  4 10  4  5 19 87  4
 10  4  5 19 75 60 14 14 19 21 19 27  4  4 14 19 97 60 41 51  6 64  5  7
 61  4 97 19]


In [5]:
# one hot encode (setting a value in a vector of zeros to one)
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    # a row for each value in 'arr', column for each lablel
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [6]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    # floor division
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [7]:
batches = get_batches(encoded, 8, 50)
print(batches)
x, y = next(batches)
print(x.size)

<generator object get_batches at 0x000002224D054B48>
400


In [8]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

# Results are shifted by one

x
[[67  6  4 41 87 46 24 19 24  7]
 [64 65 15  6  5  4 19  4  7 51]
 [65 21 46 25 19 24  7 14 52 60]
 [ 4 19 82 64 41 24 19 24 26  4]
 [60 14 41 40 19  7 87 72 24 60]
 [77 19  7 19 25  6 64 87 24  7]
 [77  5  6 25 60 41  4 19 72  6]
 [ 5 19 26  4  7  5 24 53 65 98]]

y
[[ 6  4 41 87 46 24 19 24  7 52]
 [65 15  6  5  4 19  4  7 51 26]
 [21 46 25 19 24  7 14 52 60 87]
 [19 82 64 41 24 19 24 26  4 19]
 [14 41 40 19  7 87 72 24 60 25]
 [19  7 19 25  6 64 87 24  7 60]
 [ 5  6 25 60 41  4 19 72  6 64]
 [19 26  4  7  5 24 53 65 98  6]]


In [9]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [10]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        # Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        # pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        # put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [11]:
def train(model, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        model: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    model.train()
    
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        model.cuda()
    
    counter = 0
    n_chars = len(model.chars)
    for e in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            model.zero_grad()
            
            # get the output from the model
            output, h = model(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = model.init_hidden(batch_size)
                val_losses = []
                model.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = model(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                model.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [12]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(99, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=99, bias=True)
)


In [13]:
batch_size = 64
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=1)

...Val Loss: 1.4905
Epoch: 18/20...Step: 2188...Loss: 1.1254...Val Loss: 1.4855
Epoch: 18/20...Step: 2189...Loss: 1.2149...Val Loss: 1.4823
Epoch: 18/20...Step: 2190...Loss: 1.1969...Val Loss: 1.4809
Epoch: 18/20...Step: 2191...Loss: 1.1598...Val Loss: 1.4822
Epoch: 18/20...Step: 2192...Loss: 1.1468...Val Loss: 1.4854
Epoch: 18/20...Step: 2193...Loss: 1.1301...Val Loss: 1.4902
Epoch: 18/20...Step: 2194...Loss: 1.1434...Val Loss: 1.4903
Epoch: 18/20...Step: 2195...Loss: 1.1426...Val Loss: 1.4839
Epoch: 18/20...Step: 2196...Loss: 1.1797...Val Loss: 1.4797
Epoch: 18/20...Step: 2197...Loss: 1.1234...Val Loss: 1.4808
Epoch: 18/20...Step: 2198...Loss: 1.1612...Val Loss: 1.4848
Epoch: 18/20...Step: 2199...Loss: 1.1979...Val Loss: 1.4897
Epoch: 18/20...Step: 2200...Loss: 1.1630...Val Loss: 1.4946
Epoch: 18/20...Step: 2201...Loss: 1.2291...Val Loss: 1.4965
Epoch: 18/20...Step: 2202...Loss: 1.1437...Val Loss: 1.4946
Epoch: 18/20...Step: 2203...Loss: 1.1513...Val Loss: 1.4894
Epoch: 18/20...Step:

In [14]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [15]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [16]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [19]:
print(sample(net, 1000, prime='love', top_k=2))

love

I want you all the time we could be the one

I want to say that you want to love me 

The stars wasted to the sea

And they say I love you so much

And if I can't live without you

And I don't want to lose you

I can't wait another minute 

I can't let you down

I've got a ship on the sky

I don't know what to do
I can't see the world to you all the same
I know that I want you to show
I could never leave you all the things

I can't let you down

I want to stand by my life
I want to let it go
I want you to show me 

You're the only one
I'll be there
I was made to feel this
I was meant to be

I know I wouldn't look at you

I can't wait another minute 
I want you to know
I love you to me

I would take a little bit

I know I'll be

I've been there when you are the world I would fell 
You are there when you are the only one 
I want to leave you my heart

I can't wait, it was made for me 
I want you to know
I'll be there for you
I wanna kiss your love 
All the world will live with you
